In [1]:
import jCMIP as jc
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import sys

In [2]:
CMIP6list = 'CMIP6list'
jc.generateList('6',CMIP6list)

starting models on list: 0
added ACCESS-CM2
added ACCESS-ESM1-5
added AWI-CM-1-1-HR
added AWI-CM-1-1-LR
added AWI-CM-1-1-MR
added AWI-ESM-1-1-LR
added BCC-CSM2-HR
added BCC-CSM2-MR
added BCC-ESM1
added CAMS-CSM1-0
added CAS-ESM2-0
added CESM1-CAM5-SE-HR
added CESM1-CAM5-SE-LR
added CESM2
added CESM2-FV2
added CESM2-WACCM
added CESM2-WACCM-FV2
added CIESM
added CMCC-CM2-HR4
added CMCC-CM2-SR5
added CMCC-CM2-VHR4
added CMCC-ESM2
added CNRM-CM6-1
added CNRM-CM6-1-HR
added CNRM-ESM2-1
added CanESM5
added CanESM5-CanOE
added E3SM-1-0
added E3SM-1-1
added E3SM-1-1-ECA
added EC-Earth3
added EC-Earth3-AerChem
added EC-Earth3-CC
added EC-Earth3-LR
added EC-Earth3-Veg
added EC-Earth3-Veg-LR
added EC-Earth3P
added EC-Earth3P-HR
added EC-Earth3P-VHR
added ECMWF-IFS-HR
added ECMWF-IFS-LR
added ECMWF-IFS-MR
added FGOALS-f3-H
added FGOALS-f3-L
added FGOALS-g3
added FIO-ESM-2-0
added GFDL-AM4
added GFDL-CM4
added GFDL-CM4C192
added GFDL-ESM2M
added GFDL-ESM4
added GFDL-OM4p5B
added GISS-E2-1-G
added G

In [3]:
# Add save directory to all models (only needed if you want to use this to save your data in a specific location,
# it adds that information to all the CMIP6 model objects, i.e. Clist['ACCESS-CM2'].savedir):
savedir = '/home/users/jmecking001/HACKATHON/'
jc.updateAllList(CMIP6list,savedir=(savedir))

# Directory where CMIP6 data is stored on JASMIN:
datadir6 = '/badc/cmip6/data/CMIP6/'
jc.updateAllList(CMIP6list,datadir=datadir6)

In [4]:
# Models without lon/lat properly defined:

cmip   = '6'
filename = ('CMIP' + cmip + 'list')
Clist = jc.readList(filename)
jc.updateList(filename,Model='AWI-CM-1-1-MR', Ogrid='Unstructured')
jc.updateList(filename,Model='AWI-ESM-1-1-LR',Ogrid='Unstructured')
jc.updateList(filename,Model='CAS-ESM2-0',    Ogrid='Bbl')     # incorrect lon/lat on u and v points
jc.updateList(filename,Model='CIESM',         Ogrid='Bbl')     # incorrect lon/lat on u and v point
jc.updateList(filename,Model='GFDL-CM4',      Ogrid='Cbl')     # incorrect lon/lat on u and v point
jc.updateList(filename,Model='MRI-ESM2-0',    Ogrid='Unclear') # T/U/V different sizes, need to fix
jc.updateList(filename,Model='TaiESM1-TIMCOM',Ogrid='Unclear') # No thetao, and looks like maybe A grid?

In [5]:
# Determine lon/lat, regular and grid:

cmip   = '6'
filename = ('CMIP' + cmip + 'list')
Clist = jc.readList(filename)
Models = list(Clist.keys())

var = 'thetao'

for mm in Models:
    # Check model data:
    Model = Clist[mm]
    print(mm)
    
    # Determine lon/lat:
    if Model.Olon == 'unknown':
        Files = Model.getFiles(var=var)
        nf = len(Files)
        print(nf)
        if (nf != 0):
            ncid = Dataset(Files[0],'r')
            klist = list(ncid.variables.keys())
            ncid.close()

            print(klist)
            
            for ll in range(0,len(klist)):
                if ((klist[ll] == 'lat') | (klist[ll] == 'latitude') | (klist[ll] == 'nav_lat')):
                    nlat = ll
                if ((klist[ll] == 'lon') | (klist[ll] == 'longitude') | (klist[ll] == 'nav_lon')):
                    nlon = ll

            jc.updateList(filename,Model.name,Olat=klist[nlat])
            jc.updateList(filename,Model.name,Olon=klist[nlon])

            ncid = Dataset(Files[0],'r')
            lon = ncid.variables[klist[nlon]][:]
            ncid.close()
            nd = np.squeeze((np.shape(np.shape(lon))))
            print(np.shape(lon))

            if nd == 2:
                jc.updateList(filename,Model.name,Oreg=False)
            elif nd == 1:
                jc.updateList(filename,Model.name,Oreg=True)

    Clist = jc.readList(filename)
    print(Clist[mm].Olon)
    print(Clist[mm].Olat)
    print(Clist[mm].Oreg)
    
    # Determine model grid:
    Model = Clist[mm]
    if ((Model.Ogrid == 'unknown')):
        # Find Files:
        so = Model.getFiles(var='so',EXP='*',ENS='*',vtype='Omon',gtype='gn')
        vo = Model.getFiles(var='vo',EXP='*',ENS='*',vtype='Omon',gtype='gn')
        uo = Model.getFiles(var='uo',EXP='*',ENS='*',vtype='Omon',gtype='gn')
    
        if ((len(so) != 0) & (len(vo) != 0) | (len(uo) != 0)):
            # Find mask, lat and lon:
            Tlon,Tlat = jc.Olatlon(Model,so[0],'so')
            Ulon,Ulat = jc.Olatlon(Model,uo[0],'uo')
            Vlon,Vlat = jc.Olatlon(Model,vo[0],'vo')
            
            if Model.Oreg:
                Tlon,Tlat = np.meshgrid(Tlon,Tlat)
                Vlon,Vlat = np.meshgrid(Vlon,Vlat)
                Ulon,Ulat = np.meshgrid(Ulon,Ulat)

            # Determine model grid:
            TmV_lat = Tlat[:,:]-Vlat[:,:]
            TmV_lon = Tlon[:,:]-Vlon[:,:]
            # -- => Btr
            # ++ => Bbl
            # -+ => Btl
            # -0 => Ct?
            # +0 => Cb?
            TmU_lat = Tlat[:,:]-Ulat[:,:]
            TmU_lon = Tlon[:,:]-Ulon[:,:]
            # -- => Btr
            # ++ => Bbl
            # -+ => Btl
            # 0- => C?r
            # 0+ => C?l
            VmU_lat = Vlat[:,:]-Ulat[:,:]
            VmU_lon = Vlon[:,:]-Ulon[:,:]
            # 00 => B
            # +- => Ctr
            # -- => Cbr
            # -+ => Cbl
            
            lims = 0.00001

            nn_TmV_lat = [np.sum(TmV_lat > lims), np.sum(np.abs(TmV_lat) <= lims), np.sum(TmV_lat < -lims)]
            nn_TmV_lon = [np.sum(TmV_lon > lims), np.sum(np.abs(TmV_lon) <= lims), np.sum(TmV_lon < -lims)]
            nn_TmU_lat = [np.sum(TmU_lat > lims), np.sum(np.abs(TmU_lat) <= lims), np.sum(TmU_lat < -lims)]
            nn_TmU_lon = [np.sum(TmU_lon > lims), np.sum(np.abs(TmU_lon) <= lims), np.sum(TmU_lon < -lims)]
            nn_VmU_lat = [np.sum(VmU_lat > lims), np.sum(np.abs(VmU_lat) <= lims), np.sum(VmU_lat < -lims)]
            nn_VmU_lon = [np.sum(VmU_lon > lims), np.sum(np.abs(VmU_lon) <= lims), np.sum(VmU_lon < -lims)]
            
            print('VmU lat: ' + str(nn_VmU_lat) + ', VmU lon: ' + str(nn_VmU_lon))
            print('TmU lat: ' + str(nn_TmU_lat) + ', TmU lon: ' + str(nn_TmU_lon))
            print('TmV lat: ' + str(nn_TmV_lat) + ', TmV lon: ' + str(nn_TmV_lon))
            
            if not (((np.argmax(nn_VmU_lat) == 1) & (np.argmax(nn_VmU_lon) == 1))):
                if ((np.argmax(nn_TmV_lat) == 2) & (np.argmax(nn_TmU_lon) == 2)):
                    grid = 'Ctr'   
                elif ((np.argmax(nn_TmV_lat) == 0) & (np.argmax(nn_TmU_lon) == 0)):
                    grid = 'Cbl'  
                elif ((np.argmax(nn_TmV_lat) == 2) & (np.argmax(nn_TmU_lon) == 0)):
                    grid = 'Ctl'
                elif ((np.argmax(nn_TmV_lat) == 0) & (np.argmax(nn_TmU_lon) == 2)):
                    grid = 'Cbr'
                else:
                    print('something has gone wrong')
            else:
                if ((np.argmax(nn_TmV_lat) == 2) & (np.argmax(nn_TmV_lon) == 2)):
                    grid = 'Btr'
                elif ((np.argmax(nn_TmV_lat) == 0) & (np.argmax(nn_TmV_lon) == 0)):
                    grid = 'Bbl'
                elif ((np.argmax(nn_TmV_lat) == 2) & (np.argmax(nn_TmV_lon) == 0)):
                    grid = 'Btl'
                elif ((np.argmax(nn_TmV_lat) == 0) & (np.argmax(nn_TmV_lon) == 2)):
                    grid = 'Bbr'
                else:
                    grid = 'unknown'
                    print('need to double check grid')
                    cmap = plt.get_cmap('seismic',21)

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(TmV_lat,cmap=cmap,vmin=-1,vmax=1)
                    plt.title(Model.name + ', lat: T-V')
                    plt.colorbar()

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(TmV_lon,cmap=cmap,vmin=-1,vmax=1)
                    plt.title(Model.name + ', lon: T-V')
                    plt.colorbar()

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(TmU_lat,cmap=cmap,vmin=-1,vmax=1)
                    plt.title(Model.name + ', lat: T-U')
                    plt.colorbar()

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(TmU_lon,cmap=cmap,vmin=-1,vmax=1)
                    plt.title(Model.name + ', lon: T-U')
                    plt.colorbar()

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(VmU_lat,cmap=cmap,vmin=-1,vmax=1)
                    plt.title(Model.name + ', lat: V-U')
                    plt.colorbar()

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(VmU_lon,cmap=cmap,vmin=-1,vmax=1)
                    plt.title(Model.name + ', lon: V-U')
                    plt.colorbar()

                    cmap = plt.get_cmap('seismic',3)

                    Tmask     = jc.Oread3Ddata(Model,so[0],var='so',time=0,mask=True)[0,:,:]
                    Umask     = jc.Oread3Ddata(Model,uo[0],var='uo',time=0,mask=True)[0,:,:]
                    Vmask     = jc.Oread3Ddata(Model,vo[0],var='vo',time=0,mask=True)[0,:,:]

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(Tmask+Vmask,cmap=cmap,vmin=0,vmax=2)
                    plt.title(Model.name + ', mask: T+V')
                    plt.colorbar()

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(Tmask+Umask,cmap=cmap,vmin=0,vmax=2)
                    plt.title(Model.name + ', mask: T+U')
                    plt.colorbar()

                    fig = plt.figure(figsize=(12,7))
                    plt.pcolormesh(Vmask+Umask,cmap=cmap,vmin=0,vmax=2)
                    plt.title(Model.name + ', mask: V+U')
                    plt.colorbar()

                    sys.exit()

            jc.updateList(filename,Model.name,Ogrid=grid)

    Clist = jc.readList(filename)
    print(Clist[mm].Ogrid)

ACCESS-CM2
longitude
latitude
False
Btr
ACCESS-ESM1-5
longitude
latitude
False
Btr
AWI-CM-1-1-HR
0
unknown
unknown
True
unknown
AWI-CM-1-1-LR
0
unknown
unknown
True
unknown
AWI-CM-1-1-MR
lon
lat
True
Unstructured
AWI-ESM-1-1-LR
lon
lat
True
Unstructured
BCC-CSM2-HR
longitude
latitude
False
Btl
BCC-CSM2-MR
longitude
latitude
False
Btl
BCC-ESM1
longitude
latitude
False
Btl
CAMS-CSM1-0
longitude
latitude
False
Btr
CAS-ESM2-0
lon
lat
True
Bbl
CESM1-CAM5-SE-HR
lon
lat
False
Btr
CESM1-CAM5-SE-LR
lon
lat
False
Btr
CESM2
lon
lat
False
Btr
CESM2-FV2
lon
lat
False
Btr
CESM2-WACCM
lon
lat
False
Btr
CESM2-WACCM-FV2
lon
lat
False
Btr
CIESM
longitude
latitude
False
Bbl
CMCC-CM2-HR4
longitude
latitude
False
Ctr
CMCC-CM2-SR5
longitude
latitude
False
Ctr
CMCC-CM2-VHR4
longitude
latitude
False
Ctr
CMCC-ESM2
longitude
latitude
False
Ctr
CNRM-CM6-1
lon
lat
False
Ctr
CNRM-CM6-1-HR
lon
lat
False
Ctr
CNRM-ESM2-1
lon
lat
False
Ctr
CanESM5
longitude
latitude
False
Ctr
CanESM5-CanOE
0
unknown
unknown
True
unkno